In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
import json

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
tf.keras.backend.set_floatx('float16')

In [5]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

In [6]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0,
    train_size=499
)
gpt.load_weights('./gpt_weight_pretrain/weight_fp16')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


gpt.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
)

float16 <dtype: 'float16'> True


In [7]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [8]:
ids = cbpe.encode('今天天气还行')
ids

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.632 seconds.
Prefix dict has been built successfully.


[837, 259, 497, 57, 8, 237]

In [9]:
def test_basic_logic():
    ids = cbpe.encode('今天天气还行')
    #print(ids)
    print("+" * 20)
    for i in range(10):
        output = gpt(tf.constant([ids]))
        #print(output[0].shape)
        nid = np.argmax(output[0, -1])
        ids += [nid]
        #print(i, cbpe.decode(ids))
        #print(np.argmax(output[0],axis=-1))
        print(cbpe.decode(np.argmax(output[0],axis=-1)))
        print('-' * 30)
test_basic_logic()

++++++++++++++++++++
气 不错 不错行 
------------------------------
气 不错 不错行 ,
------------------------------
气 不错 不错行 , 我
------------------------------
气 不错 不错行 , 我 就
------------------------------
气 不错 不错行 , 我 就 想
------------------------------
气 不错 不错行 , 我 就 想着
------------------------------
气 不错 不错行 , 我 就 想着 去
------------------------------
气 不错 不错行 , 我 就 想着 去 看看
------------------------------
气 不错 不错行 , 我 就 想着 去 看看 
------------------------------
气 不错 不错行 , 我 就 想着 去 看看 ,
------------------------------


In [10]:
def get_learning_rate(learning_rate=6e-4,
                      warmup_steps=20_0000,
                      decay_steps=200_0000,
                      alpha=0.0):
    def decayed_learning_rate(step=1):
        if step <= warmup_steps:
            mult = step / float(warmup_steps)
        else:
            progress = (step - warmup_steps) / (decay_steps - warmup_steps)
            mult = 0.5 * (1 + math.cos(math.pi * progress))
            mult = max(0.1, mult)
        return learning_rate * mult
    return decayed_learning_rate

In [11]:
train_recoard = json.load(open('./labeled_data/advanture_translated/truncated_advanture_train.json'))
valid_recoard = json.load(open('./labeled_data/advanture_translated/truncated_advanture_valid.json'))

In [12]:
len(train_recoard),len(valid_recoard)

(4500, 266)

In [13]:
import copy
def data_generator(stories, batch_size=4,sample_len=200,inf=False):
    while True:
        batch_data = []
        tmp_stories = copy.copy(stories)
        random.shuffle(tmp_stories)
        for i,one_story in enumerate(tmp_stories):
            story_content = one_story.to_dungeon_format()
            story_content = story_content.replace("<start>\n","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace("\n<end>","")
            story_content = story_content.replace(" ","")
            ids = cbpe.encode(story_content)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break
            
import copy
def data_generator_content(texts, batch_size=4,sample_len=200,inf=False):
    tmp_text = copy.copy(texts)
    segments = []
    while tmp_text:
        story_content = tmp_text[:10000]
        segments.append(story_content)
        tmp_text = tmp_text[10000:]
    while True:
        
        batch_data = []
        random.shuffle(segments)
        for one_story_context in segments:
            ids = cbpe.encode(one_story_context)
            while ids:
                sample = ids[:sample_len]
                ids = ids[sample_len:]
                if len(sample) < sample_len:
                    sample += [0 for i in range((sample_len - len(sample)))]
                batch_data.append(sample)
                if len(batch_data) >= batch_size:
                    yield np.asarray(batch_data)
                    batch_data = []
        if not inf:
            break
            
def turb_sentence(inputs):
    outputs = []
    for i in inputs:
        if random.random() > 0.1:
            outputs.append(i)
    return outputs
            
def data_generator_json(stories, batch_size=1,inf=False):
    while True:
        tmp_stories = copy.copy(stories)
        random.shuffle(tmp_stories)
        for i,one_story in enumerate(tmp_stories):
            previous_contents = one_story["previous"]
            previous_contents = turb_sentence(previous_contents)
            action = one_story["action"]
            content = one_story["result"]
            
            previous_contents = ''.join(previous_contents)
            content = ''.join(content)
            
            ids_prev = cbpe.encode(previous_contents + "\n")
            ids_action = cbpe.encode(">" + action + "\n")
            ids_content = cbpe.encode(content)[:300]
            
            ids_prev = ids_prev + ids_action
            
            prev_context_len = int(random.uniform(100,500))
            ids_prev = ids_prev[-prev_context_len:]
            
            overall_ids = ids_prev + ids_content
            mask = [0 for i in ids_prev] + [1 for i in ids_content]
            yield np.asarray([overall_ids]),np.asarray([mask])
            
        if not inf:
            break

In [14]:
valid_gen = data_generator_json(valid_recoard,inf=True)

In [15]:
x,m = valid_gen.__next__()

In [16]:
x.shape,m.shape

((1, 311), (1, 311))

In [17]:
cbpe.decode(x[0])

'起 。 你 觉得 有点 头晕 , 于是 你 把 自己 带到 黑暗 的 角落 休息 一下 。 您 访问 该 区域 角落 有点 暗 , 因为 自然光 几乎 无法 到达 那里 。 地板 上 似乎 也 有 一些 垃圾 。 你 进一步 移动 到 商店 的 后面 担心 周围 有 更 多 的 敌人 , 你 把 自己 拖 到 商店 的 后角 。 你 快 休息 你 无精打采 地 趴在 地上 , 试着 平稳 地 呼吸 。 要 放大 的 痛苦 众生 。 你 想 知道 这个 世界 发生 了 什么 , 是否 还有 更 多 的 敌对者 , 比如 攻击 你 的 那个 人 。 你 检查 后备箱 你 下车 , 走向 后备箱 。 你 走 回 商店 , 找到 几件 你 撕开 并 裹 在 胸前 的 衬衫 。 它 可以 帮助 您 感觉 稍微 好 一点 。 但 你 仍然 很 痛苦 。 你 弹出 后备箱 您 按 下 按钮 解锁 后备箱 。 它 缓缓 抬起 打开 。 你 发动 汽车 您 尝试 多次 启动 汽车 , 但 没有 任何 反应 。 你 想想 它 可能 需要 汽油 , 也许 他们 的 后备箱 里 有 备用 汽油 。 你 检查 地板 \n > 你 忍住 了 咳嗽 \n 当 你 看到 一个 女人 小心翼翼 地 走进 来时 , 你 把手 放在 嘴 上 。 她 看起来 并 没有 什么 害处 , 但 谁 知道 她 心里 在 想 什么 。'

In [18]:
m

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
writer = SummaryWriter('log/finetune')
n_iter = 0
train_loss = 100
val_loss = 100
for epoch in range(3):
    print(f"Epoch {epoch + 1}")
    pb = ProgressBar(len(train_recoard))
    pb.startjob()
    for x,m in data_generator_json(train_recoard):
    #for x in data_generator_content(texts[528:],sample_len=500,batch_size=2):
        n_iter += 1
        ret = gpt.train_step(
            (
                tf.constant(x[:,:-1]),
                tf.constant(x[:,1:])
            ),
            mask=m[:,1:],
        )
        writer.add_scalar("train/loss", ret["loss"].numpy(), n_iter)
        train_loss = ret["loss"].numpy()
        if n_iter % 10 == 0:
            valid_x,valid_m = valid_gen.__next__()
            ret = gpt.eval_step(
            (
                tf.constant(valid_x[:,:-1]),
                tf.constant(valid_x[:,1:])),
                mask=valid_m[:,1:],
            )
            writer.add_scalar("test/loss", ret["loss"].numpy(), n_iter)
            val_loss = ret["loss"].numpy()
        pb.info = f"tl: {train_loss} vl: {val_loss}"
        pb.complete(1)
    print()

Epoch 1
tl: 2.439453125 vl: 2.533203125 69.02 % [==================================>----------------] 3106/4500 	 used:3417s eta:1533 sss

In [ ]:
gpt.save_weights('./gpt_weight_pretrain/tmp_weight')

In [ ]:
!ls -l ./gpt_weight_pretrain/tmp_weight*

# rewrite fp16 model (weight only)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
import json

In [2]:
tf.keras.backend.set_floatx('float16')
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0,
    train_size=499
)
gpt.load_weights('./gpt_weight_pretrain/tmp_weight')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16
        
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

def test_basic_logic():
    ids = cbpe.encode('今天天气还行')
    #print(ids)
    print("+" * 20)
    for i in range(10):
        output = gpt(tf.constant([ids]))
        #print(output[0].shape)
        nid = np.argmax(output[0, -1])
        ids += [nid]
        #print(i, cbpe.decode(ids))
        #print(np.argmax(output[0],axis=-1))
        print(cbpe.decode(np.argmax(output[0],axis=-1)))
        print('-' * 30)
test_basic_logic()
        
gpt.save_weights('./gpt_weight_pretrain/tmp_weight_test')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


float16 <dtype: 'float16'> True


Loading model cost 0.559 seconds.
Prefix dict has been built successfully.


++++++++++++++++++++
好 不错行 
------------------------------
好 不错行 ,
------------------------------
好 不错行 , 
------------------------------
好 不错行 ,  
------------------------------
好 不错行 ,   
------------------------------
好 不错行 ,    
------------------------------
好 不错行 ,     
------------------------------
好 不错行 ,      
------------------------------
好 不错行 ,       
------------------------------
好 不错行 ,        
------------------------------


In [3]:
!ls -l ./gpt_weight_pretrain/tmp_weight_test*

-rw-r--r-- 1 tiger tiger 5194222858 Jul 14 10:40 ./gpt_weight_pretrain/tmp_weight_test.data-00000-of-00001
-rw-r--r-- 1 tiger tiger      34188 Jul 14 10:40 ./gpt_weight_pretrain/tmp_weight_test.index


# save model (restart kernel here)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorboardX import SummaryWriter
from tensorflow.keras.utils import multi_gpu_model
import random
tf.keras.backend.set_floatx('float16')

In [2]:
from utils.gpt2_tokenizer import GPT2Tokenizer
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [3]:
#with mirrored_strategy.scope():
gpt = GPT(
    vocab_size=30_000,
    layer_size=32,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=2560,
    num_attention_heads=32,
    attention_dropout=0.0,
    residual_dropout=0.0
)
gpt.load_weights('./gpt_weight_pretrain/tmp_weight_test')

#input_x = tf.keras.layers.Input((499,), dtype=tf.int32)
#outputs = gpt_origin(input_x)

#gpt = tf.keras.Model(inputs=input_x, outputs=outputs)
#gpt = multi_gpu_model(gpt, gpus=8)

print(tf.keras.backend.floatx(), tf.float16, tf.keras.backend.floatx() == tf.float16)
if tf.keras.backend.floatx() == tf.float16:
    for x in gpt.weights:
        assert x.dtype == tf.float16


#gpt.compile(
#    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Optimizer
#    # Loss function to minimize
#    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#    # List of metrics to monitor
#)

float16 <dtype: 'float16'> True


In [4]:
ids = cbpe.encode('今天天气不错')

for i in range(10):
    output = gpt(tf.constant([ids]))
    nid = np.argmax(output[0, -1])
    ids += [nid]
    print(i, cbpe.decode(ids))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.581 seconds.
Prefix dict has been built successfully.


0 今天天气 不错 
1 今天天气 不错 ,
2 今天天气 不错 , 我
3 今天天气 不错 , 我 想
4 今天天气 不错 , 我 想 去
5 今天天气 不错 , 我 想 去 看看
6 今天天气 不错 , 我 想 去 看看 
7 今天天气 不错 , 我 想 去 看看 。
8 今天天气 不错 , 我 想 去 看看 。 
9 今天天气 不错 , 我 想 去 看看 。 ”


In [5]:
def batch_gather(a, b):
    return tf.gather(a, b, batch_dims=1)

def penalize_used(logits, output,penalize):
    # I want to change the indices of logits wherever the index is found in output
    change_tensor = tf.zeros_like(logits, dtype=logits.dtype)
    unique = tf.unique(output[0])[0]
    ones = tf.ones_like(unique, dtype=unique.dtype)
    indices = tf.expand_dims(unique, 1)
    updates = tf.scatter_nd(indices, ones, [logits.shape[1]])
    bool_tensor = tf.expand_dims(tf.cast(updates, tf.bool), 0)
    return tf.compat.v1.where(bool_tensor, logits * penalize, logits)

def top_p_sample(logits, inputs, num_samples=1, p=0.9,k=40,temperature=0.4,penalize=0.4):
    logits = penalize_used(logits,inputs,penalize)
    batch_size, vocab_size = logits.shape
    probs = tf.nn.softmax(logits / temperature, axis=-1)
    # [batch_size, vocab_perm]
    indices = tf.argsort(probs, direction='DESCENDING')
    cumulative_probabilities = tf.math.cumsum(batch_gather(probs, indices), axis=-1, exclusive=False)
    top_k_prob = cumulative_probabilities[0][k]

    # find the top pth index to cut off. careful we don't want to cutoff everything!
    # result will be [batch_size, vocab_perm]
    p_expanded = p if isinstance(p, float) else p#[:, None]
    exclude_mask = tf.logical_not(
        tf.logical_or(
            tf.logical_and(
                cumulative_probabilities < p_expanded, 
                cumulative_probabilities < top_k_prob
            ),
            tf.range(vocab_size)[None] < 1
        )
    )

    # OPTION A - sample in the sorted space, then unsort.
    logits_to_use = batch_gather(logits, indices) - tf.cast(exclude_mask, tf.float16) * 1e4
    sample_perm = tf.random.categorical(logits=logits_to_use, num_samples=num_samples)
    sample = batch_gather(indices, sample_perm)

    return tf.cast(sample, tf.int64)

In [6]:
@tf.function
def serve(inputs):
    return gpt(inputs, kv_cache=None, use_cache=True)


@tf.function
def serve_cache(inputs, kv_cache):
    return gpt(inputs, kv_cache=kv_cache, use_cache=True)

In [7]:
serve_concrete = serve.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp")
)

layer_size = 32
attention_head = 32
embedding_size = 2560

serve_cache_concrete = serve_cache.get_concrete_function(
    tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
    tf.TensorSpec(shape=[
        layer_size, None, 2, attention_head,
        None, embedding_size // attention_head
    ], dtype=tf.float16, name="kv_cache")
)

In [8]:
r = serve_concrete(
    tf.constant([[1]], tf.int64)
)
r2 = serve_cache_concrete(
    tf.constant([[1]], tf.int64),
    r[1]
)

In [9]:
@tf.function
def sample(initial_inputs, length, p,k,temperature,penalize, num_samples=1):
    layer_size = 32
    embedding_size = 2560
    attention_head = 32

    i = tf.constant(0, dtype=tf.int64)
    initial_logits, kv_cache = serve(initial_inputs)
    inputs = top_p_sample(initial_logits[:, -1, :],initial_inputs, p=p,k=k,temperature=temperature,penalize=penalize)
    stores = tf.concat([initial_inputs, inputs], axis=1)

    def _cond(i, inputs, kv_cache, stores):
        return i < length

    def _body(i, inputs, kv_cache, stores):
        new_logits, new_kv_cache = serve_cache(inputs, kv_cache)
        
        new_inputs = top_p_sample(new_logits[:, -1, :],inputs, p=p,k=k,temperature=temperature,penalize=penalize)
        new_stores = tf.concat([stores, new_inputs], axis=-1)
        new_kv_cache = tf.concat([
            kv_cache,
            new_kv_cache
        ], axis=-2)
        new_i = i + 1
        return [new_i, new_inputs, new_kv_cache, new_stores]

    result = tf.while_loop(
        _cond, _body,
        loop_vars=[i, inputs, kv_cache, stores],
        shape_invariants=[
            tf.TensorShape(None),
            tf.TensorShape([None, None]),
            tf.TensorShape([
                layer_size, None, 2,
                attention_head, None,
                embedding_size // attention_head
            ]),
            tf.TensorShape([
                None, None
            ])
        ]
    )
    return result[-1]

In [10]:
gpt.save('./result_weights/tmp_weight', include_optimizer=False, signatures={
    'serving_default': sample.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64, name="inp"),
        tf.TensorSpec(shape=[None,], dtype=tf.int64, name="length"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="p"),
        tf.TensorSpec(shape=[], dtype=tf.int64, name="k"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="temperature"),
        tf.TensorSpec(shape=[], dtype=tf.float16, name="penalize")
    )
})

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./result_weights/tmp_weight/assets


In [11]:
ids = cbpe.encode('今天天气不错')

ret = sample(
    tf.constant([ids], dtype=tf.int64), # inp
    tf.constant(10, dtype=tf.int64), #length
    tf.constant(0.9, dtype=tf.float16), #p
    tf.constant(40, dtype=tf.int64), #k
    tf.constant(0.4, dtype=tf.float16), # temperature
    tf.constant(0.4, dtype=tf.float16) # penalize
)
print(ret)
print(cbpe.decode(ret.numpy().tolist()[0]))

tf.Tensor(
[[ 837  259  497  788    8    9  148   16  755   91 1126  375    8   12
     8]], shape=(1, 15), dtype=int64)
今天天气 不错 , 所以 我 决定 去 海边 。 


In [12]:
!ls -l ./result_weights/tmp_weight

total 21660
drwxr-xr-x 2 tiger tiger     4096 Jul  5 15:21 assets
-rw-r--r-- 1 tiger tiger 22168232 Jul 14 10:50 saved_model.pb
drwxr-xr-x 3 tiger tiger     4096 Jul 14 10:50 variables


# read deploy model (you should restart kernel and go from here)

In [1]:
%load_ext autoreload
%autoreload 2
import re
import numpy as np
import tensorflow as tf
import os
import sys
from tf2gpt.model import GPT
from utils.story_util import Story,Stories
from utils.progress_bar import ProgressBar
from tensorflow.keras.utils import multi_gpu_model
import random
from utils.gpt2_tokenizer import GPT2Tokenizer
import tensorflow_hub as hub
cbpe = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [2]:
gpt = hub.load('./result_weights/tmp_weight/')

In [3]:
def sample_gpt(tokenizer, gpt, sentence, number=1, length=20, p=0.9,k=40,temperature=0.4,penalize=0.85):
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    p = tf.constant(p, dtype=tf.float16)
    k = tf.constant(k, dtype=tf.int64)
    temperature = tf.constant(temperature, dtype=tf.float16)
    penalize = tf.constant(penalize, dtype=tf.float16)
    ret = gpt.signatures['serving_default'](
        inp=inputs, 
        length=length, 
        p=p,
        k=k,
        temperature=temperature,
        penalize=penalize)['output_0']
    return [
        tokenizer.decode(s[len(inputs[0]):]).replace(' ', '')
        for s in ret.numpy()
    ]

In [4]:
q = f'''双十一'''
ret = sample_gpt(cbpe, gpt, q, 1, 500, p=0.9,k=16,temperature=1,penalize=0.75)
for x in ret:
    print(x)
    print('-' * 20)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.558 seconds.
Prefix dict has been built successfully.


了吧。你可以看看他的回答。<eod>有人说FastestisthestupidthingFastestisthestupidthing有人说FastestisthestupidthingFastestisthestupidthingisFastestisthestupidthingFastestisthestupidthingFastestisthestupidthingFastestisthestupidthing.........FastestisthestupidthingFastestisthestupidthingFastestisthestupidthing.........FastestisthestupidthingFastestisthestupidthingFastestisthestupidthing.........Fastestisthestupidthing.........FastestisthestupidthingFastestisthestupidthingFastestisthestupidthing.........Fastestisthestupidthing...
--------------------


In [5]:
class Story():
    def __init__(self,beginning,story_max_len=200,context_len=12):
        self.story = [beginning]
        self.story_max_len = story_max_len
        self.context_len = context_len
        
    def response_quality_ok(self,response):
        if len(response) < 20:
            return False
        return True
    
    def action(self,action):
        action_str = "> 你" + action
        q = "".join(self.story[-self.context_len:]).replace("\n",'') + "\n" + action_str + "\n"
        q = q[-600:]#.replace("\n","")
        self.story.append("你" + action)
        response = ""
        for i in range(3):
            response = sample_gpt(cbpe, gpt, q, 1, 150,p=0.9,k=16,temperature=1,penalize=0.75)[0]
            response = '。'.join(response.split("。")[:-1]) + "。"
            if(self.response_quality_ok(response) == True):
                break
            else:
                print("response quality check failed,generating another one")
        
        #responsesp = response.split("。")
        #if(len(responsesp) > 2):
        #    response = ''.join(responsesp[:-1])
        #else:
        #    response = response.split(">")[0]
        #response = "。".join([:-1])
        #response = response.split(">")[0].strip()
        #response = "\n".join(response.split("\n")[:-1])
        self.story.append(response)
        
    def interactive(self):
        print("\n".join(self.story))
        while True:
            action = input("> 你")
            self.action(action)
            print(self.story[-1])

In [6]:
begin = f'''你在树林里冒险，指不定会从哪里蹦出来一些奇怪的东西，你握紧手上的手枪，希望这次冒险能够找到一些值钱的东西'''
story = Story(begin)

In [7]:
story.interactive()

你在树林里冒险，指不定会从哪里蹦出来一些奇怪的东西，你握紧手上的手枪，希望这次冒险能够找到一些值钱的东西


> 你 把手枪上膛


枪身立刻就被拉下来,你把弹匣从枪口推出来,把枪扔到地上。“现在,我们走吧。”你说。“好的,先生。”老人说。“那我们出发吧。”你说。“好的,先生。”老人说。你把手枪放进自己的口袋里,然后跟着老人继续旅行。“我们去哪里?”老人问。“去Geralt村。


> 你 走去村子里


很快,你就会到达Geralt村。村子里有个院子,院子里有个漂亮的石头房子。你走进房子,发现门廊上坐着一个很漂亮的女人,她的脸颊红润,看起来很漂亮。“嘿,丽贝卡,”你喊道,“我是来帮你的。”“谢谢,亲爱的。”“什么?”“我叫你丽贝卡,我是你的妻子。


...

> 你 走去村里的集市


现在,你去集市。这个集市在村子里,你把老人和丽贝卡留在你的领子上,然后你带着老人丽贝卡到外面的广场。你在那里看到了一个大的摊位,上面摆着面包和葡萄酒。你把老人和丽贝卡带到那里,你把老人从椅子上抱下来,然后你把她放下,开始和她交谈。


> 你 说“你觉得这个集市怎么样？”


“你认为怎么样?我想去Geralt村,我们可以去村里的大集市。你看到了一个大的布店,那里有很多布。我们可以在那里做生意。你看到了一个大的水果摊,你可以买你需要的任何水果。我们可以做到了。”“是的,”你说。“我认为我们会做到的。我们可以做到的。现在走吧,我们去Geralt村。我想看看我们会做到什么。我认为我们已经做到了。


KeyboardInterrupt: Interrupted by user